# Multi-Agent Content Creation Pipeline with CrewAI and Amazon Bedrock

This notebook implements a functional multi-agent content creation pipeline using CrewAI and Amazon Bedrock. The system performs actual web searches and uses real data to generate high-quality blog content.

## Architecture Overview

```
                ┌────────────────┐
                │                │
                │  Amazon        │
                │  Bedrock       │
                │                │
                └────────────────┘
                        ▲
                        │
                        ▼
┌───────────────────────────────────────────────┐
│                  CrewAI                       │
│  ┌──────────┐    ┌──────────┐    ┌──────────┐ │
│  │          │    │          │    │          │ │
│  │Researcher│───►│  Writer  │───►│  Editor  │ │
│  │  Agent   │    │  Agent   │    │  Agent   │ │
│  │          │    │          │    │          │ │
│  └──────────┘    └──────────┘    └──────────┘ │
│        │               │               │      │
│        ▼               ▼               ▼      │
│  ┌──────────┐    ┌──────────┐    ┌──────────┐ │
│  │ Internet │    │ Content  │    │ SEO      │ │
│  │ Search   │    │ Creation │    │ Analysis │ │
│  │ Tool     │    │ Tool     │    │ Tool     │ │
│  └──────────┘    └──────────┘    └──────────┘ │
└───────────────────────────────────────────────┘
```

## Installation and Setup

In [1]:
# Install required packages
%pip install crewai boto3 'crewai[tools]' langchain_aws duckduckgo-search -Uq


[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from pydantic import BaseModel, Field
from typing import List

# CrewAI imports
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool, tool
from crewai import LLM

# Search tools imports
from langchain_community.tools import DuckDuckGoSearchRun

# Amazon Bedrock imports
import boto3

## Configure Amazon Bedrock

Let's set up Amazon Bedrock as our LLM provider. You need to have your AWS credentials properly configured.

In [3]:
# Setting model_id and AWS region
region = "us-east-1"
model_id = "bedrock/us.amazon.nova-premier-v1:0"

# Setup credentials and create Bedrock client
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name=region,
    # Uncomment and replace with your credentials if not using IAM roles
    # aws_access_key_id=os.environ.get('AWS_ACCESS_KEY_ID'),
    # aws_secret_access_key=os.environ.get('AWS_SECRET_ACCESS_KEY')
)

# Configure the LLM
llm = LLM(
    provider="bedrock",
    model=model_id,
    temperature=0.7,
    max_tokens=4096,
    credentials={
        "aws_region_name": region
    }
)

## Define Tools for Agents

Now, let's define the tools our agents will use. We'll implement actual search functionality with DuckDuckGo for comprehensive research and an SEO analysis tool.

In [5]:
# Create a DuckDuckGo search tool
@tool("Internet Search")
def search_internet(query: str) -> str:
    """Search the internet for information on a specific query or topic.
    
    Args:
        query: The search query string.
    
    Returns:
        str: Search results as text.
    """
    search_tool = DuckDuckGoSearchRun()
    results = search_tool.run(query)
    return f"Search results for '{query}':\n\n{results}"

# SEO Analysis Tool
class SEOAnalysisInput(BaseModel):
    """Input schema for SEO analysis tool."""
    content: str = Field(..., description="The content to analyze for SEO optimization")
    target_keywords: List[str] = Field(..., description="List of target keywords to check for in the content")

class SEOAnalysisTool(BaseTool):
    """Tool for analyzing content for SEO optimization."""
    name: str = "SEO Content Analyzer"
    description: str = "Analyzes content for SEO optimization and provides recommendations"
    args_schema: type[BaseModel] = SEOAnalysisInput
    
    def _run(self, content: str, target_keywords: List[str]) -> str:
        # Calculate basic metrics
        word_count = len(content.split())
        
        # Calculate keyword density
        keyword_counts = {}
        content_lower = content.lower()
        
        for keyword in target_keywords:
            count = content_lower.count(keyword.lower())
            density = round(count / word_count * 100, 2) if word_count > 0 else 0
            keyword_counts[keyword] = {
                "count": count,
                "density": density
            }
        
        # Calculate readability metrics
        sentences = max(1, content.count('.') + content.count('!') + content.count('?'))
        avg_words_per_sentence = word_count / sentences
        
        # Generate recommendations
        recommendations = []
        
        if word_count < 800:
            recommendations.append("Increase content length to at least 800 words for better SEO performance")
        
        for keyword, data in keyword_counts.items():
            if data["count"] == 0:
                recommendations.append(f"Add the keyword '{keyword}' to your content")
            elif data["density"] < 0.5:
                recommendations.append(f"Increase the usage of '{keyword}' (current density: {data['density']}%)")
            elif data["density"] > 2.5:
                recommendations.append(f"Reduce the usage of '{keyword}' to avoid keyword stuffing (current density: {data['density']}%)")
        
        # Format response
        response = f"# SEO Content Analysis\n\n"
        response += f"## Overview\n"
        response += f"- Word Count: {word_count} words ({word_count < 800 and 'too short' or word_count > 2000 and 'lengthy' or 'good length'})\n"
        response += f"- Average Sentence Length: {round(avg_words_per_sentence, 1)} words\n\n"
        
        response += f"## Keyword Analysis\n"
        for keyword, data in keyword_counts.items():
            response += f"- '{keyword}': {data['count']} occurrences ({data['density']}% density)\n"
        
        response += f"\n## Recommendations\n"
        if not recommendations:
            response += "- Content is well-optimized for the target keywords.\n"
        else:
            for rec in recommendations:
                response += f"- {rec}\n"
        
        return response

## Define Specialized Agents

Now let's create our three specialized agents: Researcher, Writer, and Editor.

In [6]:
# Create the Researcher Agent
researcher = Agent(
    role="Research Specialist",
    goal="Conduct thorough research on topics and provide comprehensive, accurate information",
    backstory="""You are an expert researcher with a talent for finding the most relevant and up-to-date 
    information on any topic. You're known for your ability to gather comprehensive data, verify sources, 
    and organize information logically. Your research always includes key statistics, expert opinions, 
    and real-world examples to provide a complete picture of the topic.""",
    verbose=True,
    allow_delegation=False,
    tools=[search_internet],
    llm=llm
)

# Create the Writer Agent
writer = Agent(
    role="Content Writer",
    goal="Create engaging, informative content based on research findings",
    backstory="""You are a talented content writer with years of experience creating engaging blog posts. 
    You excel at turning research into compelling narratives that educate and entertain. 
    Your writing is always clear, structured, and tailored to the target audience. 
    You know how to naturally incorporate keywords while maintaining a conversational tone.""",
    verbose=True,
    allow_delegation=False,  # Disable delegation to avoid errors
    tools=[],  # The writer uses the LLM's capabilities for content creation
    llm=llm
)

# Create the Editor Agent
editor = Agent(
    role="Content Editor & SEO Specialist",
    goal="Refine and optimize content for readability and SEO performance",
    backstory="""You are a meticulous editor and SEO specialist. You have a keen eye for detail 
    and can transform good content into excellent content. You understand SEO best practices 
    and know how to optimize content for search engines without sacrificing readability or user experience. 
    You ensure content is error-free, well-structured, and strategically optimized for its target keywords.""",
    verbose=True,
    allow_delegation=False,
    tools=[SEOAnalysisTool()],
    llm=llm
)

## Define Tasks for Each Agent

Now let's define the specific tasks for our content creation pipeline. We'll choose a topic and target keywords for our blog post.

In [7]:
# Define blog post topic and target keywords
blog_topic = "Sustainable Gardening Practices for Urban Homes"
target_keywords = ["sustainable gardening", "urban gardening", "eco-friendly gardening", "composting", "water conservation"]

# Task 1: Research the topic thoroughly
research_task = Task(
    description=f"""
    Research the topic of "{blog_topic}" comprehensively. Your research should include:
    
    1. Latest trends and innovations in {blog_topic}
    2. Best practices and techniques that are accessible for urban dwellers
    3. Expert opinions and data-backed information
    4. Common challenges and practical solutions
    5. Real-world examples and success stories
    6. Resources or tools that would be helpful to readers
    
    Use the Internet Search tool to gather diverse and current information.
    Focus on content that includes the target keywords: {', '.join(target_keywords)}
    
    Organize your research findings in a structured format that will be helpful for the content writer.
    Include specific data points, statistics, and quotable insights when available.
    
    Return a comprehensive, organized research document with clear sections.
    """,
    agent=researcher,
    expected_output="A comprehensive research brief with organized, factual information about sustainable gardening practices for urban homes, including trends, best practices, expert insights, and real-world examples."
)

# Task 2: Write engaging blog content based on research
writing_task = Task(
    description=f"""
    Using the research provided, create an engaging and informative blog post about "{blog_topic}".
    
    The research findings are as follows:
    {{research_task.output}}
    
    Your blog post should:
    1. Start with an engaging introduction that hooks the reader and establishes the importance of the topic
    2. Include a logical structure with clear headings and subheadings
    3. Provide practical, actionable advice that urban dwellers can implement
    4. Incorporate real examples, success stories, or case studies
    5. Naturally integrate the target keywords: {', '.join(target_keywords)}
    6. End with a compelling conclusion and call-to-action
    
    The tone should be conversational yet informative, accessible to beginners while still valuable to those with some gardening experience.
    
    Aim for approximately 1,200-1,500 words in total.
    
    Return a complete, well-structured blog post in markdown format.
    """,
    agent=writer,
    context=[research_task],
    expected_output="A well-structured, engaging blog post about sustainable gardening practices for urban homes that effectively utilizes the research findings and naturally incorporates the target keywords."
)

# Task 3: Edit and optimize the content for SEO
editing_task = Task(
    description=f"""
    Review, edit, and optimize the following blog post about "{blog_topic}".
    
    Content to edit:
    {{writing_task.output}}
    
    Your editing tasks:
    
    1. Improve the overall structure and flow
    2. Enhance readability with appropriate formatting (headings, bullet points, short paragraphs)
    3. Optimize for SEO using the target keywords: {', '.join(target_keywords)}
    4. Check factual accuracy and ensure proper attribution of sources when needed
    5. Refine the writing style for clarity and engagement
    6. Create a compelling meta title and meta description
    
    First, use the SEO Content Analyzer tool to evaluate and identify areas for improvement.
    Then, apply your improvements to create the final version.
    
    Return the final, optimized blog post in markdown format, including your suggested meta title and description at the top.
    """,
    agent=editor,
    context=[writing_task],
    expected_output="A fully optimized, polished blog post with enhanced structure, readability, and SEO optimization, along with a recommended meta title and description."
)

## Create and Run the Content Creation Crew

Now we'll create and run our content creation crew with the defined agents and tasks.

In [8]:
# Create the content creation crew
content_crew = Crew(
    agents=[researcher, writer, editor],
    tasks=[research_task, writing_task, editing_task],
    verbose=True,  # Set to True for detailed logs
    process=Process.sequential  # Execute tasks in sequence
)

# Run the crew
result = content_crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0d359dd8-0a5d-4bf9-a716-7a97cc804a83                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 0582c114-bff9-4ee9-acc0-29b083ca6615
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 0582c114-bff9-4ee9-acc0-29b083ca6615
       Status: Executing Task...
    └── 🤖 Agent: Research Specialist
            Status: In Progress

# Agent: Research Specialist
## Task: 
    Research the topic of "Sustainable Gardening Practices for Urban Homes" comprehensively. Your research should include:
    
    1. Latest trends and innovations in Sustainable Gardening Practices for Urban Homes
    2. Best practices and techniques that are accessible for urban dwellers
    3. Expert opinions and data-backed information
    4. Common challenges and practical solutions
    5. Real-world examples and success stories
    6. Resources or tools that would be helpful to readers
    
    Use the Internet Search tool to gather diverse and current information.
    Focus on content that includes the target keywords: sustainable gardening, urban gardening, eco-friendly gardening, composting, water conservation
    
    Organize your research findings in a structured format that will be helpful for the content writer.
    Include specific data points, statistics, and quotable insights when available.
    
    Return a comprehensive, organ

🤖 Agent: Research Specialist
    Status: In Progress
└── 🧠 Thinking...

...



# Agent: Content Editor & SEO Specialist
## Final Answer: 
```markdown
# Urban Sustainable Gardening: Eco-Friendly Practices for City Dwellers

**Meta Title:** Urban Sustainable Gardening: Eco-Friendly Tips & Practices (59 chars)  
**Meta Description:** Discover sustainable gardening techniques for urban homes. Learn composting, water conservation, and eco-friendly practices to transform your space. (155 chars)

---

Transform your tiny balcony or rooftop into a lush oasis with **sustainable gardening**! Urban environments aren't just for concrete - they're perfect for creating **eco-friendly gardens**. With [35% of households now gardening](https://www.usda.gov/), there's never been a better time to join the green revolution.

## Why Urban Gardening Is Essential
Urban gardens provide more than beauty:
- **Temperature Reduction**: Lower local temps by 2°C (Dr. Elena Vasquez, Urban Ecology Institute)
- **Energy Savings**: Cut cooling costs by 10-15% (EPA)
- **Carbon Sequestration**: A

🚀 Crew: crew
├── 📋 Task: 0582c114-bff9-4ee9-acc0-29b083ca6615
│      Assigned to: Research Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Research Specialist
│           Status: ✅ Completed
├── 📋 Task: 46dc4efe-5d63-4d1e-b1bc-43307f3cfe53
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 07d2dcb8-efd0-470b-a266-c3c5c04c3293
       Status: Executing Task...
    └── 🤖 Agent: Content Editor & SEO Specialist
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 0582c114-bff9-4ee9-acc0-29b083ca6615
│      Assigned to: Research Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Research Specialist
│           Status: ✅ Completed
├── 📋 Task: 46dc4efe-5d63-4d1e-b1bc-43307f3cfe53
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 07d2dcb8-efd0-470b-a266-c3c5c04c3293
       Assigned to: Content Editor & SEO Specialist
       Status: ✅ Completed
    └── 🤖 Agent: Content Editor & SEO Specialist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 07d2dcb8-efd0-470b-a266-c3c5c04c3293                                                                     │
│  Agent: Content Editor & SEO Specialist                                                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0d359dd8-0a5d-4bf9-a716-7a97cc804a83                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Display the Final Blog Post and Save Results

In [21]:
# Display the final edited blog post
print("\n" + "="*80)
print("FINAL BLOG POST CONTENT")
print("="*80 + "\n")
print(result)


FINAL BLOG POST CONTENT

```markdown
# Urban Sustainable Gardening: Eco-Friendly Practices for City Dwellers

**Meta Title:** Urban Sustainable Gardening: Eco-Friendly Tips & Practices (59 chars)  
**Meta Description:** Discover sustainable gardening techniques for urban homes. Learn composting, water conservation, and eco-friendly practices to transform your space. (155 chars)

---

Transform your tiny balcony or rooftop into a lush oasis with **sustainable gardening**! Urban environments aren't just for concrete - they're perfect for creating **eco-friendly gardens**. With [35% of households now gardening](https://www.usda.gov/), there's never been a better time to join the green revolution.

## Why Urban Gardening Is Essential
Urban gardens provide more than beauty:
- **Temperature Reduction**: Lower local temps by 2°C (Dr. Elena Vasquez, Urban Ecology Institute)
- **Energy Savings**: Cut cooling costs by 10-15% (EPA)
- **Carbon Sequestration**: Absorb 1.2 tons CO₂/acre annually (C

## Display Individual Agent Outputs (Optional)

Let's also look at the output from each stage of the process.

In [22]:
# Display research findings
print("\n" + "="*80)
print("RESEARCH FINDINGS")
print("="*80 + "\n")
print(research_task.output)


RESEARCH FINDINGS

**Sustainable Gardening Practices for Urban Homes: Comprehensive Research**

---

### **1. Latest Trends & Innovations (2023)**
- **Smart Gardening Systems**: IoT-enabled sensors monitor soil moisture and automate watering schedules, reducing water waste by up to 50% (Smart Garden Activator, 2023).  
- **Vertical Farming**: Space-efficient hydroponic systems like AeroGardens allow year-round herb production in 1 sq ft of countertop space.  
- **Regenerative Practices**: No-dig gardening and biochar amendments improve soil health by 30% (Rodale Institute, 2023).  
- **Edible Landscaping**: Replacing lawns with fruit trees and berry bushes increases food yield by 2-3x per square meter (Urban Agriculture Magazine, 2022).

---

### **2. Best Practices & Techniques**
- **Container Gardening**: Use self-watering planters with reservoirs to reduce evaporation losses by 70%.  
- **Composting**: Vermicomposting (worm bins) converts kitchen scraps into nutrient-rich soil in 3

In [23]:
# Display initial blog draft
print("\n" + "="*80)
print("INITIAL BLOG DRAFT")
print("="*80 + "\n")
print(writing_task.output)


INITIAL BLOG DRAFT

```markdown
# Sustainable Gardening Practices for Urban Homes: Transform Your Space Today

Did you know that a small balcony or rooftop can become a thriving oasis of greenery? **Sustainable gardening** isn’t just for suburban lawns—urban dwellers can turn concrete jungles into eco-friendly havens. With 35% of households now gardening (USDA), there's no better time to join the movement. Let’s explore how you can implement **eco-friendly gardening** practices right at home.

## Why Urban Gardening Matters
Urban gardens do more than beautify spaces. They reduce local temperatures by 2°C (Dr. Elena Vasquez), lower energy costs by 10-15%, and sequester 1.2 tons of CO₂ per acre (Carbon Trust). Plus, they boost biodiversity—urban plots support 5x more bee species than parks!

---

## **Latest Trends in Sustainable Urban Gardening**
### 1. **Smart Gardening Systems**
IoT-enabled sensors (like Smart Garden Activator) automate watering, cutting waste by 50%. These systems m

## Conclusion

This CrewAI content pipeline demonstrates how multiple specialized AI agents can collaborate to create high-quality content. The process follows a logical workflow:

1. The Researcher agent gathers comprehensive information from the web
2. The Writer agent transforms that research into engaging content
3. The Editor agent optimizes the content for readability and SEO

The result is a polished, SEO-optimized blog post ready for publication, created entirely through the orchestrated collaboration of AI agents.